In [1056]:
import numpy as np
import imutils
import cv2

In [1057]:
def four_point_transform(gr, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    maxWidth+=(maxWidth*12)//10
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(gr, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [1058]:
def order_points(pts):
    pts=np.array(pts)
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

In [1298]:
plate = cv2.imread("13.png")
gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)

In [1299]:
rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)

In [1300]:
cv2.imshow("cropped img",blackhat)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1317]:
squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
light = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, squareKern)
light = cv2.threshold(light, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

In [1302]:
cv2.imshow("cropped img",light)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1303]:
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F,dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
gradX = gradX.astype("uint8")

In [1304]:
cv2.imshow("cropped img",gradX)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1305]:
gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
thresh = cv2.threshold(gradX, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

In [1306]:
cv2.imshow("cropped img",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1307]:
thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

In [1308]:
cv2.imshow("cropped img",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1309]:
thresh = cv2.bitwise_and(thresh, thresh, mask=light)
thresh = cv2.dilate(thresh, None, iterations=2)
thresh = cv2.erode(thresh, None, iterations=1)

In [1310]:
cv2.imshow("cropped img",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1311]:
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:3]

In [1312]:
contours=cnts
screenCnt=None
img_area=(thresh.shape[0])*(thresh.shape[1])

In [1313]:
for cnt in contours:
    print(cv2.contourArea(cnt),img_area)
    if cv2.contourArea(cnt)>((img_area*2)//10):
        print(1)
        epsilon = 0.025*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        print(len(approx))
        if(len(approx)>4 and len(approx)<7):
            diff=0.001
            epsilon = (0.025+diff)*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            while(len(approx)!=4):
                diff=diff+0.001
                epsilon = (0.025+diff)*cv2.arcLength(cnt,True)
                approx = cv2.approxPolyDP(cnt,epsilon,True)
        if(len(approx)==3):
            diff=0.001
            epsilon = (0.025-diff)*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            while(len(approx)!=4):
                diff=diff+0.001
                epsilon = (0.025-diff)*cv2.arcLength(cnt,True)
                approx = cv2.approxPolyDP(cnt,epsilon,True)        
                
        if len(approx) == 4:
            screenCnt = approx
            break

4749.0 13797
1
5


In [1314]:
screenCnt[0][0]

array([18, 16], dtype=int32)

In [1010]:
if contours:
    areas = [cv2.contourArea(c) for c in contours]

    # index of the largest contour in the area
    # array
    max_index = np.argmax(areas)  

    cnt = contours[max_index]
    #print(cnt)
    epsilon = 0.025*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    rect = cv2.minAreaRect(approx)
    box=cv2.boxPoints(rect)
    box=np.int0(box)
    #plate= cv2.drawContours(plate,[box],0,(0,255,0),3)
    mult = 1
    print(rect)
    print(box)
    imagee=four_point_transform(gray, box)
#     W = rect[1][0]
#     H = rect[1][1]

#     Xs = [i[0] for i in box]
#     Ys = [i[1] for i in box]
#     x1 = min(Xs)
#     x2 = max(Xs)
#     y1 = min(Ys)
#     y2 = max(Ys)

#     rotated = False
#     angle = rect[2]

#     if angle < -45:
#         angle+=90
#         rotated = True

#     center = (int((x1+x2)/2), int((y1+y2)/2))
#     size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

#     M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

#     cropped = cv2.getRectSubPix(gray, size, center)    
#     cropped = cv2.warpAffine(cropped, M, size)
    
#     croppedW = W if not rotated else H 
#     croppedH = H if not rotated else W

#     croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))

# if((croppedRotated.shape[0]-croppedRotated.shape[1])>10):
#     croppedRotated = cv2.rotate(croppedRotated, cv2.cv2.ROTATE_90_CLOCKWISE)
cv2.imshow("final img",plate)
cv2.imshow("cropped img",imagee)
cv2.waitKey(0)
cv2.destroyAllWindows()

((51.157806396484375, 50.433387756347656), (102.42021942138672, 42.69152069091797), -32.347442626953125)
[[ 19  95]
 [ -3  59]
 [ 82   5]
 [105  41]]


In [1315]:
imagee=four_point_transform(plate, list([screenCnt[0][0],screenCnt[1][0],screenCnt[2][0],screenCnt[3][0]]))

In [1316]:
cv2.imshow("cropped img",imagee)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1214]:
cv2.imwrite("16.png",imagee)

True

In [1128]:
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)